<a href="https://colab.research.google.com/github/Nov05/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/blob/master/cfpd_consumer_complaints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-06-24

In [9]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('drive')
root = 'drive/My Drive/'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data  
https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD

In [14]:
# CAUTION: this dataset is 719 MB in size.

# this doesn't work for this large file (seems that it would for file under 50 MB)
# url = "https://drive.google.com/uc?id=1giGHkApdv3ehRI7uw4LoLhGT6rN_uR0r"

# this works
url = "https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(url)

# path = "content/Consumer_Complaints.csv"
# path = "content/drive/My Drive/data/cfpd_consumer_complaints/Consumer_Complaints.csv"
# df = pd.read_csv(path)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
print(df.shape) # (1315204, 18)
df.head()

(1315204, 18)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,06/22/2019,Debt collection,Payday loan debt,Took or threatened to take negative or legal a...,Sued you without properly notifying you of law...,NaN,NaN,Nationwide Recovery Group LLC,OR,971XX,NaN,NaN,Web,06/22/2019,Closed with monetary relief,Yes,NaN,3283266
1,06/22/2019,Debt collection,Medical debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,Armada Corp.,WA,98801,NaN,NaN,Web,06/22/2019,Closed with explanation,Yes,NaN,3283648
2,06/21/2019,Debt collection,Medical debt,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Credit Bureau of Carbon County,TX,NaN,NaN,NaN,Web,06/21/2019,In progress,Yes,NaN,3282841
3,06/21/2019,Debt collection,Medical debt,False statements or representation,Attempted to collect wrong amount,NaN,Company has responded to the consumer and the ...,"HCFS Health Care Financial Services, Inc.",FL,330XX,NaN,NaN,Web,06/21/2019,Closed with explanation,Yes,NaN,3282839
4,06/21/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,"Diversified Consultants, Inc.",GA,30349,NaN,NaN,Web,06/21/2019,In progress,Yes,NaN,3282748


In [18]:
# 
df.describe(exclude=np.number)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?
count,1315204,1315204,1080038,1315204,780025,390137,466682,1315204,1294645,1195363,179973,710785,1315204,1315204,1315197,1315204,768501
unique,2760,18,76,167,218,373093,10,5321,63,22771,3,4,6,2709,8,2,2
top,09/08/2017,Mortgage,Credit reporting,Incorrect information on your report,Information belongs to someone else,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"EQUIFAX, INC.",CA,300XX,Servicemember,Consent provided,Web,09/08/2017,Closed with explanation,Yes,No
freq,3553,280699,237756,144871,65757,993,327233,120369,180085,7601,87241,390459,972452,3387,1019448,1282533,620123


In [29]:
# check NAs count and percentage for each column
nas = np.array(df.isna().sum())
pd.DataFrame([df.columns, nas, nas/df.shape[0] * 100]).T

,0,1,2
0,Date received,0,0
1,Product,0,0
2,Sub-product,235166,17.8806
3,Issue,0,0
4,Sub-issue,535179,40.6917
5,Consumer complaint narrative,925067,70.3364
6,Company public response,848522,64.5164
7,Company,0,0
8,State,20559,1.56318
9,ZIP code,119841,9.11197


In [30]:
# check product
df['Product'].value_counts()

Mortgage                                                                        280699
Debt collection                                                                 250487
Credit reporting, credit repair services, or other personal consumer reports    242344
Credit reporting                                                                140432
Credit card                                                                      89190
Bank account or service                                                          86206
Student loan                                                                     52430
Credit card or prepaid card                                                      50656
Checking or savings account                                                      43438
Consumer Loan                                                                    31605
Vehicle loan or lease                                                            12032
Money transfer, virtual currency, or money 